In [ ]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
files_ns = np.random.choice(range(0, 500), size=(10,), replace=False)
data = []

for n in files_ns:
    with h5py.File(
        "/home/de64/scratch/de64/sync_folder/2021-05-27_lDE18_20x_run_1/mVenus/hdf5/hdf5_"
        + str(n)
        + ".hdf5",
        "r",
    ) as infile:
        data.append(infile["mCherry"][np.random.randint(0, 5)].astype(float))
data = np.array(data)[:, 1000:2000, :1500]
# data[data<500] = np.NaN

In [ ]:
data > 1000

In [ ]:
plt.imshow(data[18])

In [ ]:
## "Estimating the ﬂat-ﬁeld S by low rank and sparse decomposition"

data.shape
I_sort = np.sort(data, axis=0)
I_sort = I_sort.reshape(I_sort.shape[0], -1).T

In [ ]:
# Y (0) = 0;S(0) = 0;B(0) = 0;R(0) = 0;µ(0) > 0;ρ > 1;k = 0
lambda_s = 1000.0

W = np.ones(I_sort.shape, dtype=float)
eps = 0.00001
Y = np.zeros(I_sort.shape, dtype=float)
S = np.zeros(I_sort.shape[0], dtype=float)  ## not sure if one-dim correct here
B = np.zeros(I_sort.shape[1], dtype=float)
IR = np.zeros(I_sort.shape, dtype=float)
u = 1.0
rho = 1.5
k = 0

tol = 0.001

In [ ]:
plt.imshow(IB[:, 0].reshape(data.shape[1], data.shape[2]))

In [ ]:
plt.imshow(IR[:, 0].reshape(data.shape[1], data.shape[2]))

In [ ]:
plt.imshow((IB + IR)[:, 0].reshape(data.shape[1], data.shape[2]))

In [ ]:
data.shape

In [ ]:
# Sf = np.fft.fft(S).real
# IB = np.multiply.outer(S,B)
# theta_one = Sf+np.fft.fft(np.nanmean((I_sort-IB-IR+(Y/u)),axis=1)).real
# Sf_plus_1 = np.sign(theta_one)*np.maximum(np.abs(theta_one)-(lambda_s/u),0.)

In [ ]:
np.isnp.nanmean((I_sort - IB - IR + (Y / u)), axis=1)

In [ ]:
l1_minimization(I_sort, data.shape)

In [ ]:
def l1_minimization(I_sort, data_shape, W):
    lambda_s = 400.0

    Y = np.zeros(I_sort.shape, dtype=float)
    S = np.zeros(I_sort.shape[0], dtype=float)  ## not sure if one-dim correct here
    B = np.zeros(I_sort.shape[1], dtype=float)
    IR = np.zeros(I_sort.shape, dtype=float)
    u = 12.5
    rho = 1.5
    k = 0
    maxiter = 10
    tol = 0.001

    I_sort_norm = np.sum(np.square(I_sort))

    Ls = []
    Ts = []
    while True:
        #     Sf = np.fft.fft(S)
        #     IB = np.multiply.outer(S,B)
        #     theta_one = Sf+np.fft.fft(np.mean((I_sort-IB-IR+(Y/u)),axis=1))
        #     Sf_plus_1 = np.sign(theta_one)*np.maximum(theta_one-(lambda_s/u),0.)
        #     S_plus_1 = np.abs(np.fft.ifft(Sf_plus_1))

        #     if k > 0:
        #         W_plus_1 = 1/(np.abs(IR/IB)+eps)

        Sf = np.fft.fft2(S.reshape(data_shape[1], data_shape[2])).real.reshape(-1)
        IB = np.multiply.outer(S, B)
        theta_one = Sf + np.fft.fft(np.mean((I_sort - IB - IR + (Y / u)), axis=1)).real
        Sf_plus_1 = np.sign(theta_one) * np.maximum(
            np.abs(theta_one) - (lambda_s / u), 0.0
        )

        #     plt.plot(theta_one)
        #     plt.show()

        S_plus_1 = np.fft.ifft2(
            Sf_plus_1.reshape(data_shape[1], data_shape[2])
        ).real.reshape(-1)

        #     print("Block1")
        #     print(IR)
        #     print(IB)
        #     print(1/(IR/IB))

        IB_step_1 = np.multiply.outer(S_plus_1, B)
        theta_two = I_sort - IB_step_1 + (Y / u)
        IR_plus_1 = np.sign(theta_two) * np.maximum(np.abs(theta_two) - (W / u), 0.0)
        B_plus_1 = np.mean((I_sort - IR_plus_1), axis=0) / np.mean((I_sort - IR_plus_1))
        IB_plus_1 = np.multiply.outer(S_plus_1, B_plus_1)
        Y_plus_1 = Y + u * (I_sort - IB_plus_1 - IR_plus_1)
        u_plus_1 = rho * u

        #     print("Block2")
        #     print(S_plus_1)
        #     print(B_plus_1)

        #     if k > 0:
        #         W_plus_1 = 1/(np.abs(IR/IB)+eps)
        #         W = W_plus_1

        S = S_plus_1
        IR = IR_plus_1
        B = B_plus_1
        IB = IB_plus_1
        Y = Y_plus_1
        u = u_plus_1
        #     if k > 0:
        #         W = W_plus_1

        #     print(S)
        #     print(IR)
        #     print(B)
        T1 = np.linalg.norm((W * IR), ord=1)
        T2 = lambda_s * np.linalg.norm(
            np.fft.fft2(S.reshape(data_shape[1], data_shape[2])), ord=1
        )
        T3 = np.sum(Y * (I_sort - IB - IR))
        current_norm = np.sum(np.square(I_sort - IB - IR))
        T4 = u * current_norm
        L = T1 + T2 + T3 + T4

        Ls.append(L)
        Ts.append([T1, T2, T3, T4])

        stopCriterion = current_norm / I_sort_norm
        print(stopCriterion)

        k += 1

        if stopCriterion < tol or k >= maxiter:
            break

    Ts = np.array(Ts)
    plt.plot(Ls)
    plt.show()
    plt.plot(Ts)
    plt.show()
    return S, IB, IR

In [ ]:
eps = 0.00001
W = np.ones(I_sort.shape, dtype=float)

S, IB, IR = l1_minimization(I_sort, data.shape, W)

for i in range(3):
    W = 1.0 / (np.abs(IR / IB) + eps)
    S, IB, IR = l1_minimization(I_sort, data.shape, W)

In [ ]:
plt.imshow(S.reshape(data.shape[1], data.shape[2]))

In [ ]:
plt.plot(Ts[:, 1])

In [ ]:
# plt.plot(Ts[:,0])
plt.plot(Ts[:, 1])
plt.plot(Ts[:, 2])
plt.plot(Ts[:, 3])

In [ ]:
plt.plot(Ts[:, 0])

In [ ]:
np.linalg.norm((W * IR), ord=1)

In [ ]:
lambda_s * np.linalg.norm(np.fft.fft(S), ord=1)

In [ ]:
np.sum(Y * (I_sort - IB - IR))

In [ ]:
u * (np.sum(np.square(I_sort - IB - IR)))

In [ ]:
np.linalg.norm(np.fft.fft(np.mean(I_sort, axis=1) / 66000).real, ord=1) / 800

In [ ]:
np.fft.fft(np.mean(I_sort, axis=1) / 60000).real

In [ ]:
norm_S = S.reshape(data.shape[1], data.shape[2]) / np.mean(S)

In [ ]:
plt.hist(norm_S.flatten())

In [ ]:
fig = plt.figure(figsize=(16, 24))
plt.imshow(norm_S, cmap="Greys_r")
plt.show()

In [ ]:
plt.plot(np.mean(norm_S, axis=0))

In [ ]:
plt.plot(np.mean(norm_S, axis=1))

In [ ]:
IB_step_1

In [ ]:
S_plus_1.shape

In [ ]:
IB.shape